In [1]:
import os
import pydicom
import numpy as np
from PIL import Image
import openslide as openslide
import scripts.conversion
from PIL.GimpGradientFile import linear
%load_ext autoreload
%autoreload 2
from tqdm import tqdm
from IPython.display import clear_output


In [6]:
scripts.conversion.Conversiontopng.convert_dicom_svs_to_png(
    input_folder="data\\Yale_HER2_cohort\\SVS_positive",
    output_folder="output_dsfalse",
    svs_level=0,
    auto_downsample=False,
    max_images=1,
)


Found 1 files to process.


Converting files:   0%|          | 0/1 [00:00<?, ?file/s]

Converting files: 100%|██████████| 1/1 [00:53<00:00, 53.64s/file]


--- Conversion Summary ---
[SVS] Saved: Her2Pos_Case_45.svs


In [ ]:
# Import WSI training functions
import scripts.train as wsi_train

# Display available functions
print("🧠 WSI Training Functions Available:")
print("1. wsi_train.create_trainer() - Create a trainer instance")
print("2. wsi_train.quick_train() - Quick training with default settings")
print("3. wsi_train.WSITrainer - Full trainer class for advanced use")
print("4. wsi_train.WholeSlideDataset - Dataset class for patch extraction")

In [ ]:
# Quick Training Example
# Uncomment and modify the parameters below to start training

# trainer = wsi_train.quick_train(
#     data_dir="data\\Yale_HER2_cohort",  # Your data directory
#     model_name='resnet50',              # or 'efficientnet_b0'
#     num_classes=2,                      # Number of classes
#     num_epochs=5,                       # Number of training epochs
#     patch_size=224,                     # Size of patches
#     patches_per_slide=20,               # Patches per slide
#     batch_size=8                        # Batch size
# )

In [ ]:
# Advanced Training Example
# For more control over training parameters

# # Step 1: Create trainer
# trainer = wsi_train.create_trainer(
#     model_name='resnet50',
#     num_classes=2,
#     checkpoint_dir='checkpoints'
# )

# # Step 2: Prepare data
# train_size, val_size = trainer.prepare_data(
#     data_dir="data\\Yale_HER2_cohort",
#     patch_size=224,
#     patches_per_slide=50,
#     batch_size=16,
#     val_split=0.2,
#     num_workers=2
# )

# # Step 3: Setup training parameters
# trainer.setup_training(
#     learning_rate=0.001,
#     weight_decay=1e-4
# )

# # Step 4: Load existing checkpoint (if any)
# trainer.load_checkpoint()

# # Step 5: Start training
# trainer.train(num_epochs=10)

In [ ]:
# Training Visualization and Evaluation
# Use these after training is complete

# # Plot training progress
# if 'trainer' in locals():
#     trainer.plot_training_progress()

# # Save the trained model
# if 'trainer' in locals():
#     trainer.save_checkpoint(
#         trainer.current_epoch, 
#         trainer.best_val_acc, 
#         is_best=True
#     )
#     print(f"Model saved! Best accuracy: {trainer.best_val_acc:.4f}")

# # Load a saved model for inference
# model, class_names, checkpoint = wsi_train.load_model_for_inference('checkpoints/best_model.pth')
# print(f"Loaded model with classes: {class_names}")